In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [23]:
data = pd.read_csv('transactions.csv')

In [24]:
# Encode categorical features
label_encoder = LabelEncoder()
data["encoded_category"] = label_encoder.fit_transform(data["category"])

In [25]:
# Normalize numerical features
scaler = MinMaxScaler()
data['normalized_amount'] = scaler.fit_transform(data['amount'].values.reshape(-1, 1))

In [26]:
X = data[['normalized_amount']]
y = data['encoded_category']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# Build LSTM Model
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

In [38]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [39]:
# Train the model
model.fit(X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=200, batch_size=32)

Epoch 1/200
11/11 [==============================] - 9s 9ms/step - loss: 2.1957 - accuracy: 0.1485
Epoch 2/200
11/11 [==============================] - 0s 9ms/step - loss: 2.1909 - accuracy: 0.2121
Epoch 3/200
11/11 [==============================] - 0s 9ms/step - loss: 2.1847 - accuracy: 0.2061
Epoch 4/200
11/11 [==============================] - 0s 10ms/step - loss: 2.1773 - accuracy: 0.2121
Epoch 5/200
11/11 [==============================] - 0s 9ms/step - loss: 2.1644 - accuracy: 0.2212
Epoch 6/200
11/11 [==============================] - 0s 9ms/step - loss: 2.1482 - accuracy: 0.2364
Epoch 7/200
11/11 [==============================] - 0s 9ms/step - loss: 2.1235 - accuracy: 0.2576
Epoch 8/200
11/11 [==============================] - 0s 9ms/step - loss: 2.0926 - accuracy: 0.2606
Epoch 9/200
11/11 [==============================] - 0s 9ms/step - loss: 2.0578 - accuracy: 0.2606
Epoch 10/200
11/11 [==============================] - 0s 12ms/step - loss: 2.0198 - accuracy: 0.2636
Epoch 1

In [40]:
test_loss, test_acc = model.evaluate(X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test)
print('Test Accuracy:', test_acc)

3/3 [==============================] - 2s 6ms/step - loss: 0.7306 - accuracy: 0.6867
Test Accuracy: 0.6867470145225525


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [42]:
# Step 1: Feature Engineering
# Reshape the input features
X_train_rf = X_train.values.reshape((X_train.shape[0], X_train.shape[1]))
X_test_rf = X_test.values.reshape((X_test.shape[0], X_test.shape[1]))

In [43]:
# Step 2: Model Training
# Initialize and train the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train)

RandomForestClassifier(random_state=42)

In [44]:
# Step 3: Model Evaluation
# Predict on the testing data
y_pred_rf = rf_model.predict(X_test_rf)

In [45]:
# Evaluate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print('Random Forest Classifier Accuracy:', accuracy_rf)

Random Forest Classifier Accuracy: 0.7590361445783133


In [54]:
# Prediction on new data
new_data = pd.DataFrame({'amount': [200]})
new_data['normalized_amount'] = scaler.transform(new_data['amount'].values.reshape(-1, 1))
prediction = model.predict(new_data['normalized_amount'].values.reshape((1, 1, 1))).argmax()
category = label_encoder.inverse_transform([prediction])[0]  # Ensure input is a 1D array
new_data['category'] = category
print(new_data)

1/1 [==============================] - 0s 37ms/step
   amount  normalized_amount    category
0     200           0.011356  Healthcare
